### Feature based classification

In this file, we aim to classify the recorded activities using traditional methods, including preprocessing, feature extraction, and classification. Activities include: Walking, Sitting Down, Standing Up, Picking up an Object, Drinking Water, Falling

Importing relevant libraries:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from scipy.signal import find_peaks
from scipy.stats import entropy

Loading dataset:

In [3]:
x = np.load('datasets/scaled_spec_resampled_array.npy')
y = np.load('datasets/labels_array.npy')-1
subjects = np.load('datasets/subjects_array.npy')
activities = ['Walking', 'Sitting Down', 'Standing Up', 'Picking up an Object', 'Drinking Water', 'Falling']

print(f"Spectrograms (X) shape: {x.shape}")
print(f"Labels (Y) shape: {y.shape}")
print(f"Subjects shape: {subjects.shape}")

num_samples = x.shape[0]
num_classes = len(np.unique(y))
print(f"Number of samples: {num_samples}")
print(f"Number of classes: {num_classes}")

Spectrograms (X) shape: (1754, 2048, 80)
Labels (Y) shape: (1754,)
Subjects shape: (1754,)
Number of samples: 1754
Number of classes: 6


#### Preprocessing and Feature extraction

Setting parameters:

In [4]:
nfft = 2048
fs = 128000
f_lo = -64000.0     # Obtained from dataset_creation.ipynb
f_hi = 63937.5

freq_axis = np.linspace(f_lo, f_hi, nfft)

Denoising and feature extraction based on this paper:

Y. Kim and H. Ling, "Human Activity Classification Based on Micro-Doppler Signatures Using a Support Vector Machine," in IEEE Transactions on Geoscience and Remote Sensing, vol. 47, no. 5, pp. 1328-1337, May 2009

Features extracted:

(1) Torso Doppler frequency: Indicates the speed of the human subject.

(2) Total bandwidth (BW) of the Doppler signal: Relates to the speed of limb motions.

(3) Offset of the total Doppler signal: Measures asymmetry between forward and backward limb motions.

(4) BW without micro-Dopplers: Represents the Doppler bandwidth of the torso alone.

(5) Normalized standard deviation (STD) of the Doppler signal strength: Related to the dynamic range of the motion.

(6) Period of limb motion: Corresponds to the swing rate of arms and legs.


Also added 2 other features:

(7) Entropy: How dispersed is the energy across frequencies.

(8) High envelope standard deviation: Can indicate different types of motion. For instance, it could be used to identify the spectrogram peaks when picking up an object

In [5]:
def denoise_and_extract_features(frequencies, spectrogram, noise_threshold):
    # Initialize feature variables
    torso_doppler_frequency = []
    total_bw_doppler = []
    offset_total_doppler = []
    bw_without_micro_dopplers = []
    normalized_std_doppler = []
    entropies = []
    high_envelope = []
    low_envelope = []
    # Denoise spectrogram
    spec_denoised = np.where(spectrogram < noise_threshold, noise_threshold, spectrogram)
    for t_index in range(spec_denoised.shape[1]):

        column = spec_denoised[:, t_index]
        
        if np.all(column == noise_threshold):   # If the entire column is noise
            high_freq = 0
            low_freq = 0
            peak_freq = 0
            total_bw = 0
            offset_total = 0
            normalized_std = 0
            entr = 0
        else:   # If the column contains signal
            high_freq = frequencies[column > noise_threshold][-1]
            low_freq = frequencies[column > noise_threshold][0]
            # Torso Doppler Frequency (1)
            peak_freq = frequencies[np.argmax(column)]
            # Total Bandwidth of the Doppler Signal (2)
            total_bw = high_freq-low_freq
            # Offset of the Total Doppler (3)
            offset_total = (high_freq + low_freq) / 2
            # Normalized STD of the Doppler Signal Strength (5)
            std_doppler = np.std(column)
            mean_doppler = np.mean(column)
            normalized_std = std_doppler / mean_doppler if mean_doppler != 0 else 0
            # New feature: Entropy (7)
            entr = entropy(column)

        high_envelope.append(high_freq)
        low_envelope.append(low_freq)
        torso_doppler_frequency.append(peak_freq)
        total_bw_doppler.append(total_bw)
        offset_total_doppler.append(offset_total)
        normalized_std_doppler.append(normalized_std)
        entropies.append(entr)

    # Bandwidth Without Micro-Dopplers (4)
    bw_without_micro_dopplers = np.mean(np.array(sorted(high_envelope)[-5:]) - np.array(sorted(low_envelope)[:5]))

    # Period of Limb Motion (6)
    peaks, _ = find_peaks(high_envelope, height=np.nanmean(high_envelope))
    peak_intervals = np.diff(peaks)
    periods = peak_intervals / fs
    mean_period = np.mean(periods) if len(periods) > 0 else 0

    # New feature: Peak frequency standard deviation (8)
    peak_freq_variation = np.std(high_envelope)

    return [np.mean(torso_doppler_frequency),
            np.mean(total_bw_doppler),
            np.mean(offset_total_doppler),
            bw_without_micro_dopplers,
            np.mean(normalized_std_doppler),
            mean_period,
            np.mean(entropies),
            peak_freq_variation]

Now we extract these features from the dataset:

In [6]:
noise_threshold = -83  # dBm (based on the paper's noise threshold)
features = []
for i in range(num_samples):
    features.append(denoise_and_extract_features(freq_axis, x[i,:,:], noise_threshold))
features = np.array(features)
print(features.shape)

(1754, 8)


Let's find out the most important features by measuring the mutual information among the features:

In [7]:
from sklearn.feature_selection import mutual_info_classif

mutual_infos = mutual_info_classif(X=features, y=y)
for feature, mi in enumerate(mutual_infos, start=1):
    print(f"MI between target and feature {feature}: {mi:.4f}")

MI between target and feature 1: 0.9180
MI between target and feature 2: 0.4555
MI between target and feature 3: 0.6908
MI between target and feature 4: 0.3872
MI between target and feature 5: 0.5082
MI between target and feature 6: 0.1052
MI between target and feature 7: 0.6996
MI between target and feature 8: 0.2877


Feature number 6 (Period of Limb Motion) appears to be the least important, as it has the lowest mutual information score with the label vector among all the features. Given its low contribution, we can consider removing this feature from our dataset.

In [8]:
features = features[:, [0, 1, 2, 3, 4, 6, 7]]  # Remove period of limb motion (feature 6 - index 5)

#### Dataset Splitting

Now that we have extracted the features, it is time to split the dataset. Due to the limited dataset size, we chose to perform 5-fold cross-validation to obtain a more accurate estimate of each model's performance.

We also note that each of these 5 folds contains activities performed by different subjects, with no overlap across folds. In other words, in each iteration, the test set contains activities from subjects not used in training. This approach simulates a realistic scenario, as in the real world, the subjects in the test set are not seen by the model during training.

It's also important to note that the number of activities performed varies across subjects, i.e. some subjects have performed more repetitions than others (see code block below). We account for this variation when splitting the data, resulting in 5 roughly equal folds.

In [9]:
# Number of activities different per subject
unique, counts = np.unique(subjects, return_counts=True)
subject_counts = dict(zip(unique, counts))
sorted_subject_counts = dict(sorted(subject_counts.items(), key=lambda item: item[1], reverse=True))
for key, value in sorted_subject_counts.items():
    print(f"Subject {key} performed {value} activities")

Subject 8 performed 36 activities
Subject 14 performed 36 activities
Subject 57 performed 36 activities
Subject 30 performed 34 activities
Subject 53 performed 34 activities
Subject 28 performed 33 activities
Subject 29 performed 33 activities
Subject 31 performed 33 activities
Subject 32 performed 33 activities
Subject 33 performed 33 activities
Subject 34 performed 33 activities
Subject 36 performed 33 activities
Subject 37 performed 33 activities
Subject 38 performed 33 activities
Subject 39 performed 33 activities
Subject 40 performed 33 activities
Subject 41 performed 33 activities
Subject 43 performed 33 activities
Subject 44 performed 33 activities
Subject 45 performed 33 activities
Subject 46 performed 33 activities
Subject 47 performed 33 activities
Subject 50 performed 33 activities
Subject 51 performed 33 activities
Subject 52 performed 33 activities
Subject 55 performed 33 activities
Subject 56 performed 33 activities
Subject 35 performed 32 activities
Subject 54 performed 

Split the data in roughly 5 equal folds, ensuring that each subject is present in only in one fold:

In [10]:
gkf = GroupKFold(n_splits=5)
splits = list(gkf.split(x, y, groups=subjects))

# Verification:
for i, (train_index, test_index) in enumerate(splits):
    print(f"\nFold {i+1}:")
    print(f"Length of training set: {len(train_index)} samples")
    print(f"Length of testing set (fold): {len(test_index)} samples")
    print(f"Unique subjects in fold {i+1}: {np.unique(subjects[test_index])}")


Fold 1:
Length of training set: 1398 samples
Length of testing set (fold): 356 samples
Unique subjects in fold 1: [ 4 16 20 23 25 28 32 38 42 46 54 57 58 60 67]

Fold 2:
Length of training set: 1401 samples
Length of testing set (fold): 353 samples
Unique subjects in fold 2: [ 5  8  9 19 21 31 35 39 45 49 52 61 64 68 72]

Fold 3:
Length of training set: 1405 samples
Length of testing set (fold): 349 samples
Unique subjects in fold 3: [ 2  3  7 11 14 18 26 29 37 44 55 65 70 71]

Fold 4:
Length of training set: 1406 samples
Length of testing set (fold): 348 samples
Unique subjects in fold 4: [ 6 12 17 22 34 36 43 47 48 53 56 62 63 69]

Fold 5:
Length of training set: 1406 samples
Length of testing set (fold): 348 samples
Unique subjects in fold 5: [ 1 10 13 15 24 27 30 33 40 41 50 51 59 66]


Let's move on to the classification. We try KNN, Random Forests, XGBoost and SVM

#### Classification

##### KNN Classifier

In [10]:
k_values = np.arange(1,11)
accuracies = np.zeros((5, len(k_values)))
precisions = np.zeros((5, len(k_values), num_classes))
recalls = np.zeros((5, len(k_values), num_classes))
f1_scores = np.zeros((5, len(k_values), num_classes))

for i, (train_index, test_index) in enumerate(splits):
    x_train, x_test = features[train_index], features[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(f"Fold {i+1}...")
    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(x_train, y_train)
        y_pred = knn.predict(x_test)
        accuracies[i,k-1] = accuracy_score(y_test, y_pred)
        precisions[i,k-1,:] = precision_score(y_test, y_pred, average=None)
        recalls[i,k-1,:] = recall_score(y_test, y_pred, average=None)
        f1_scores[i,k-1,:] = f1_score(y_test, y_pred, average=None)
print("Done!\n")

mean_accuracies = np.mean(accuracies, axis=0)
mean_precisions = np.mean(precisions, axis=0)
mean_recalls = np.mean(recalls, axis=0)
mean_f1_scores = np.mean(f1_scores, axis=0)

# Print the scores for each k
for i in range(len(k_values)):
    print(f"For k = {k_values[i]}:")
    print(f"KNN mean accuracy = {100*mean_accuracies[i]:.2f}%")
    print("KNN precisions per activity:")
    print([f"{activities[j]}: {100*mean_precisions[i,j]:.2f}%" for j in range(num_classes)])
    print("KNN recalls per activity:")
    print([f"{activities[j]}: {100*mean_recalls[i,j]:.2f}%" for j in range(num_classes)])
    print("KNN F1 scores per activity:")
    print([f"{activities[j]}: {100*mean_f1_scores[i,j]:.2f}%" for j in range(num_classes)])
    print()

Fold 1...
Fold 2...
Fold 3...
Fold 4...
Fold 5...
Done!

For k = 1:
KNN mean accuracy = 75.55%
KNN precisions per activity:
['Walking: 97.54%', 'Sitting Down: 89.11%', 'Standing Up: 85.36%', 'Picking up an Object: 50.87%', 'Drinking Water: 50.05%', 'Falling: 87.67%']
KNN recalls per activity:
['Walking: 98.10%', 'Sitting Down: 85.58%', 'Standing Up: 80.72%', 'Picking up an Object: 53.11%', 'Drinking Water: 50.67%', 'Falling: 90.35%']
KNN F1 scores per activity:
['Walking: 97.79%', 'Sitting Down: 87.19%', 'Standing Up: 82.82%', 'Picking up an Object: 51.82%', 'Drinking Water: 50.23%', 'Falling: 88.87%']

For k = 2:
KNN mean accuracy = 74.63%
KNN precisions per activity:
['Walking: 96.74%', 'Sitting Down: 81.04%', 'Standing Up: 78.60%', 'Picking up an Object: 49.33%', 'Drinking Water: 51.52%', 'Falling: 94.18%']
KNN recalls per activity:
['Walking: 99.38%', 'Sitting Down: 90.38%', 'Standing Up: 87.46%', 'Picking up an Object: 61.48%', 'Drinking Water: 29.65%', 'Falling: 81.78%']
KNN F1 s

##### Random Forest Classifier

In [11]:
tree_values = [10, 100, 250, 500, 1000]
accuracies = np.zeros((5, len(tree_values)))
precisions = np.zeros((5, len(tree_values), num_classes))
recalls = np.zeros((5, len(tree_values), num_classes))
f1_scores = np.zeros((5, len(tree_values), num_classes))
for i, (train_index, test_index) in enumerate(splits):
    x_train, x_test = features[train_index], features[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(f"Fold {i+1}...")
    for j, trees in enumerate(tree_values):
        rf = RandomForestClassifier(n_estimators=trees)
        rf.fit(x_train, y_train)
        y_pred = rf.predict(x_test)
        accuracies[i,j] = accuracy_score(y_test, y_pred)
        precisions[i,j,:] = precision_score(y_test, y_pred, average=None, labels=np.unique(y_pred))
        recalls[i,j,:] = recall_score(y_test, y_pred, average=None, labels=np.unique(y_pred))
        f1_scores[i,j,:] = f1_score(y_test, y_pred, average=None, labels=np.unique(y_pred))
print("Done!\n")

mean_accuracies = np.mean(accuracies, axis=0)
mean_precisions = np.mean(precisions, axis=0)
mean_recalls = np.mean(recalls, axis=0)
mean_f1_scores = np.mean(f1_scores, axis=0)

# Print the scores for each tree number
for i in range(len(tree_values)):
    print(f"For {tree_values[i]} trees:")
    print(f"RF mean accuracy = {100*mean_accuracies[i]:.2f}%")
    print("RF precisions per activity:")
    print([f"{activities[j]}: {100*mean_precisions[i,j]:.2f}%" for j in range(num_classes)])
    print("RF recalls per activity:")
    print([f"{activities[j]}: {100*mean_recalls[i,j]:.2f}%" for j in range(num_classes)])
    print("RF F1 scores per activity:")
    print([f"{activities[j]}: {100*mean_f1_scores[i,j]:.2f}%" for j in range(num_classes)])
    print()

Fold 1...
Fold 2...
Fold 3...
Fold 4...
Fold 5...
Done!

For 10 trees:
RF mean accuracy = 82.63%
RF precisions per activity:
['Walking: 99.37%', 'Sitting Down: 94.14%', 'Standing Up: 88.49%', 'Picking up an Object: 63.14%', 'Drinking Water: 63.79%', 'Falling: 90.37%']
RF recalls per activity:
['Walking: 99.38%', 'Sitting Down: 93.91%', 'Standing Up: 89.73%', 'Picking up an Object: 64.00%', 'Drinking Water: 59.41%', 'Falling: 92.83%']
RF F1 scores per activity:
['Walking: 99.37%', 'Sitting Down: 93.94%', 'Standing Up: 88.68%', 'Picking up an Object: 63.44%', 'Drinking Water: 61.42%', 'Falling: 91.13%']

For 100 trees:
RF mean accuracy = 83.93%
RF precisions per activity:
['Walking: 99.37%', 'Sitting Down: 93.94%', 'Standing Up: 91.73%', 'Picking up an Object: 66.45%', 'Drinking Water: 64.34%', 'Falling: 91.77%']
RF recalls per activity:
['Walking: 99.69%', 'Sitting Down: 93.27%', 'Standing Up: 90.39%', 'Picking up an Object: 65.65%', 'Drinking Water: 63.23%', 'Falling: 95.45%']
RF F1 sc

##### XGBoost Classifier

In [12]:
# XGBoost
tree_values = [10, 100, 250, 500, 1000]
accuracies = np.zeros((5, len(tree_values)))
precisions = np.zeros((5, len(tree_values), num_classes))
recalls = np.zeros((5, len(tree_values), num_classes))
f1_scores = np.zeros((5, len(tree_values), num_classes))
for i, (train_index, test_index) in enumerate(splits):
    x_train, x_test = features[train_index], features[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(f"Fold {i+1}...")
    for j, trees in enumerate(tree_values):
        xgb = XGBClassifier(n_estimators=trees)
        xgb.fit(x_train, y_train)
        y_pred = xgb.predict(x_test)
        accuracies[i,j] = accuracy_score(y_test, y_pred)
        precisions[i,j] = precision_score(y_test, y_pred, average=None)
        recalls[i,j] = recall_score(y_test, y_pred, average=None)
        f1_scores[i,j] = f1_score(y_test, y_pred, average=None)
print("Done!\n")

mean_accuracies = np.mean(accuracies, axis=0)
mean_precisions = np.mean(precisions, axis=0)
mean_recalls = np.mean(recalls, axis=0)
mean_f1_scores = np.mean(f1_scores, axis=0)

# Print the scores for each tree number
for i in range(len(tree_values)):
    print(f"For {tree_values[i]} trees:")
    print(f"XGBoost mean accuracy = {100*mean_accuracies[i]:.2f}%")
    print("XGBoost precisions per activity:")
    print([f"{activities[j]}: {100*mean_precisions[i,j]:.2f}%" for j in range(num_classes)])
    print("XGBoost recalls per activity:")
    print([f"{activities[j]}: {100*mean_recalls[i,j]:.2f}%" for j in range(num_classes)])
    print("XGBoost F1 scores per activity:")
    print([f"{activities[j]}: {100*mean_f1_scores[i,j]:.2f}%" for j in range(num_classes)])
    print()

Fold 1...
Fold 2...
Fold 3...
Fold 4...
Fold 5...
Done!

For 10 trees:
XGBoost mean accuracy = 84.50%
XGBoost precisions per activity:
['Walking: 99.36%', 'Sitting Down: 94.53%', 'Standing Up: 91.62%', 'Picking up an Object: 67.45%', 'Drinking Water: 65.30%', 'Falling: 92.69%']
XGBoost recalls per activity:
['Walking: 100.00%', 'Sitting Down: 93.91%', 'Standing Up: 91.03%', 'Picking up an Object: 62.71%', 'Drinking Water: 68.73%', 'Falling: 93.95%']
XGBoost F1 scores per activity:
['Walking: 99.68%', 'Sitting Down: 94.20%', 'Standing Up: 91.20%', 'Picking up an Object: 64.94%', 'Drinking Water: 66.90%', 'Falling: 92.91%']

For 100 trees:
XGBoost mean accuracy = 84.10%
XGBoost precisions per activity:
['Walking: 99.69%', 'Sitting Down: 94.18%', 'Standing Up: 91.78%', 'Picking up an Object: 66.34%', 'Drinking Water: 64.30%', 'Falling: 93.52%']
XGBoost recalls per activity:
['Walking: 99.38%', 'Sitting Down: 92.01%', 'Standing Up: 92.30%', 'Picking up an Object: 64.30%', 'Drinking Water: 

##### SVM Classifier

Note: 'poly' and 'sigmoid' kernels perform a lot worse than the 'linear' and 'rbf' kernels, so they have been omitted

In [13]:
kernels = ['linear', 'rbf']
accuracies = np.zeros((5, len(kernels)))
precisions = np.zeros((5, len(kernels), num_classes))
recalls = np.zeros((5, len(kernels), num_classes))
f1_scores = np.zeros((5, len(kernels), num_classes))
for i, (train_index, test_index) in enumerate(splits):
    x_train, x_test = features[train_index], features[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(f"Fold {i+1}...")
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    for j, kernel in enumerate(kernels):
        svc = SVC(kernel=kernels[j], random_state=42)
        svc.fit(x_train_scaled, y_train)
        y_pred = svc.predict(x_test_scaled)
        accuracies[i,j] = accuracy_score(y_test, y_pred)
        precisions[i,j,:] = precision_score(y_test, y_pred, average=None)
        recalls[i,j,:] = recall_score(y_test, y_pred, average=None)
        f1_scores[i,j,:] = f1_score(y_test, y_pred, average=None)
print("Done!")

mean_accuracies = np.mean(accuracies, axis=0)
mean_precisions = np.mean(precisions, axis=0)
mean_recalls = np.mean(recalls, axis=0)
mean_f1_scores = np.mean(f1_scores, axis=0)

# Print the scores for each kernel
for i in range(len(kernels)):
    print(f"For {kernels[i]} kernel:")
    print(f"SVM mean accuracy = {100*mean_accuracies[i]:.2f}%")
    print("SVM precisions per activity:")
    print([f"{activities[j]}: {100*mean_precisions[i,j]:.2f}%" for j in range(num_classes)])
    print("SVM recalls per activity:")
    print([f"{activities[j]}: {100*mean_recalls[i,j]:.2f}%" for j in range(num_classes)])
    print("SVM F1 scores per activity:")
    print([f"{activities[j]}: {100*mean_f1_scores[i,j]:.2f}%" for j in range(num_classes)])
    print()

Fold 1...
Fold 2...
Fold 3...
Fold 4...
Fold 5...
Done!
For linear kernel:
SVM mean accuracy = 81.30%
SVM precisions per activity:
['Walking: 97.20%', 'Sitting Down: 94.65%', 'Standing Up: 92.37%', 'Picking up an Object: 55.80%', 'Drinking Water: 58.68%', 'Falling: 94.90%']
SVM recalls per activity:
['Walking: 98.75%', 'Sitting Down: 94.56%', 'Standing Up: 91.66%', 'Picking up an Object: 61.75%', 'Drinking Water: 51.97%', 'Falling: 93.42%']
SVM F1 scores per activity:
['Walking: 97.95%', 'Sitting Down: 94.59%', 'Standing Up: 91.87%', 'Picking up an Object: 58.54%', 'Drinking Water: 55.00%', 'Falling: 93.86%']

For rbf kernel:
SVM mean accuracy = 82.96%
SVM precisions per activity:
['Walking: 98.14%', 'Sitting Down: 95.46%', 'Standing Up: 92.29%', 'Picking up an Object: 62.35%', 'Drinking Water: 61.65%', 'Falling: 93.85%']
SVM recalls per activity:
['Walking: 98.44%', 'Sitting Down: 92.95%', 'Standing Up: 90.38%', 'Picking up an Object: 61.72%', 'Drinking Water: 65.14%', 'Falling: 92.29